In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

import bs4
import re

## For a player

In [2]:
options = webdriver.EdgeOptions()
# options.add_argument('headless')
options.add_argument('inprivate')
driver = webdriver.Edge(options= options)

In [16]:
driver.get('https://qlstats.net/player/63562')

### Cookie screen

In [3]:
def is_cookie_screen(driver : selenium.webdriver):
    '''A function to check if the given webpage is the 'accept cookies' screen.
    Regex matches the body of the '''
    element = driver.find_element(By.TAG_NAME, value = 'body')
    cookie_screen = re.compile(r'.*(To continue using qlstats, you need to agree to the use of cookies.\nAgree).*')
    if cookie_screen.search(element.text) is None:
        return False
    return True


In [4]:
#press the button
if is_cookie_screen(driver):
    try:
        button = driver.find_element(By.TAG_NAME, 'button')
        button.click()
    except:
        "Cookies could not be accepted, please recheck"

In [33]:
# reached the player page, press the overall button
button = driver.find_element(By.CLASS_NAME, 'tab-overall')
button.click()

### Scraping the player page

In [34]:
from bs4 import BeautifulSoup

In [35]:
soup = BeautifulSoup(driver.page_source, 'lxml')


In [48]:
# finding player name
player_name = soup.select_one('h2', style = 'display:inline-block').text
print('Player name:\n{}'.format(player_name))

Player name:
A S U R A


In [49]:
# parse information from the p tab
soup.select_one('p')

<p>
      
      Region: Asia
      <br/>Player ID: 63562
      <br/>Steam ID: <a href="http://steamcommunity.com/profiles/76561197968261050/" target="_blank">76561197968261050</a>
<br/>Joined: <span class="abstime" data-epoch="1455556904" title="Joined about 7 years ago">2016-02-15   22:51:44</span>
<br/>Status: active
    </p>

In [63]:
# parse winrate and elo info
soup.select('div', class_ = 'row tab-pane active')[1]

<div class="navbar-header">
<a class="navbar-toggle" data-target=".navbar-collapse" data-toggle="collapse">
<span class="icon-bar"></span>
<span class="icon-bar"></span>
<span class="icon-bar"></span>
</a>
<a class="navbar-brand" href="/">
<img alt="QL Logo" height="32" src="/static/css/img/menu_icon.png" style="float:left;margin-top:-5px;margin-right:10px;-webkit-filter: drop-shadow(0px 5px 5px grey);" width="32"/>
        QLStats
      </a>
</div>

In [ ]:
soup.findAll('table', class_= 'table table-hover table-condensed')

### Shut down the browser

In [64]:
# driver.close()